In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime as dt
from scipy.spatial import distance
import scipy.stats as stats

%matplotlib inline

In [2]:
TIME_SAMPLE_FREQ = '5s'

In [3]:
df = pd.read_csv(".\\pipelines\\gen\\user_1_sergey\\broadcasts.data", sep = ';', index_col = False, header = None, low_memory = False, \
                 names = ['timestamp', 'action', 'data', 'package', 'scheme', 'type'])

In [4]:
df.head()

,timestamp,action,data,package,scheme,type
0,12.03.2021_09:26:34.041,android.intent.action.BATTERY_CHANGED,NaN,NaN,NaN,NaN
1,12.03.2021_09:26:34.043,android.net.wifi.p2p.STATE_CHANGED,NaN,NaN,NaN,NaN
2,12.03.2021_09:26:34.052,android.intent.action.SERVICE_STATE,NaN,NaN,NaN,NaN
3,12.03.2021_09:26:34.045,android.telephony.action.DEFAULT_SMS_SUBSCRIPT...,NaN,NaN,NaN,NaN
4,12.03.2021_09:26:34.052,android.media.ACTION_SCO_AUDIO_STATE_UPDATED,NaN,NaN,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179889 entries, 0 to 179888
Data columns (total 6 columns):
timestamp    179889 non-null object
action       179889 non-null object
data         817 non-null object
package      74397 non-null object
scheme       817 non-null object
type         0 non-null float64
dtypes: float64(1), object(5)
memory usage: 8.2+ MB


In [6]:
drop_actions = [
  'android.net.wifi.SCAN_RESULTS',
  'android.bluetooth.device.action.FOUND',
  'android.bluetooth.adapter.action.DISCOVERY_STARTED',
  'android.bluetooth.adapter.action.DISCOVERY_FINISHED'
]

df = df[~df['action'].str.contains('|'.join(drop_actions))]
df = df.drop(['package', 'scheme', 'type'], axis = 1)

In [7]:
df['timestamp'] = df['timestamp'].apply(lambda x: dt.strptime(x, '%d.%m.%Y_%H:%M:%S.%f'))

In [8]:
df.index = pd.DatetimeIndex(df.timestamp)
df = df.sort_index()

In [9]:
#pd.value_counts(df['action'])

In [10]:
power_events_df = df[df['action'].str.contains('|'.join( \
                            ['android.intent.action.SCREEN_ON', \
                             'android.intent.action.SCREEN_OFF']))]

In [14]:
power_events_df = power_events_df.append(
    {
        'timestamp' : df.iloc[0][0], 
        'action' : 'android.intent.action.SCREEN_ON', 
        'data' : df.iloc[0][2]
    }, ignore_index=True)

In [18]:
power_events_df.index = pd.DatetimeIndex(power_events_df.timestamp)
power_events_df = power_events_df.sort_index()

In [34]:
with open(".\\pipelines\\gen\\user_1_sergey\\power.data", 'a') as f:
    time_array = []
    for i in range(len(power_events_df)):
        action = power_events_df['action'].iloc[i]
        timestamp = power_events_df['timestamp'].iloc[i]

        if action == "android.intent.action.SCREEN_ON":
            on_time = timestamp
        else:
            time_array.append([on_time, timestamp])
            
    for on, off in time_array:
        if (off - on).total_seconds() <= 60 * 60:
            f.write(str(on) + ';' + str(off) + '\n')

In [39]:
time_array = []
with open(".\\pipelines\\gen\\user_1_sergey\\power.data", 'r') as f:
    lines = f.readlines()
    for line in lines:
        tmp = line.split(';')
        time_array.append([tmp[0], tmp[1].replace('\n', '')])

In [41]:
df_parts = []
for on, off in time_array:
    df_parts.append(df.loc[pd.Timestamp(on) : pd.Timestamp(off)])

In [44]:
pd.concat(df_parts)

,timestamp,action,data
timestamp,,,
2021-03-08 15:15:01.225,2021-03-08 15:15:01.225,android.intent.action.BATTERY_CHANGED,NaN
2021-03-08 15:15:01.227,2021-03-08 15:15:01.227,android.intent.action.SERVICE_STATE,NaN
2021-03-08 15:15:01.229,2021-03-08 15:15:01.229,android.net.wifi.WIFI_STATE_CHANGED,NaN
2021-03-08 15:15:01.229,2021-03-08 15:15:01.229,android.net.conn.TETHER_STATE_CHANGED,NaN
2021-03-08 15:15:01.229,2021-03-08 15:15:01.229,android.net.wifi.STATE_CHANGE,NaN
2021-03-08 15:15:01.229,2021-03-08 15:15:01.229,android.intent.action.SIM_STATE_CHANGED,NaN
2021-03-08 15:15:01.229,2021-03-08 15:15:01.229,android.net.conn.INET_CONDITION_ACTION,NaN
2021-03-08 15:15:01.229,2021-03-08 15:15:01.229,android.net.nsd.STATE_CHANGED,NaN
2021-03-08 15:15:01.230,2021-03-08 15:15:01.230,android.net.conn.CONNECTIVITY_CHANGE,NaN


In [38]:
df.loc[pd.Timestamp('2021-03-08 15:15:01.225') : pd.Timestamp('2021-03-08 15:15:01.234')]

,timestamp,action,data
timestamp,,,
2021-03-08 15:15:01.225,2021-03-08 15:15:01.225,android.intent.action.BATTERY_CHANGED,NaN
2021-03-08 15:15:01.227,2021-03-08 15:15:01.227,android.intent.action.SERVICE_STATE,NaN
2021-03-08 15:15:01.229,2021-03-08 15:15:01.229,android.net.wifi.WIFI_STATE_CHANGED,NaN
2021-03-08 15:15:01.229,2021-03-08 15:15:01.229,android.net.conn.TETHER_STATE_CHANGED,NaN
2021-03-08 15:15:01.229,2021-03-08 15:15:01.229,android.net.wifi.STATE_CHANGE,NaN
2021-03-08 15:15:01.229,2021-03-08 15:15:01.229,android.intent.action.SIM_STATE_CHANGED,NaN
2021-03-08 15:15:01.229,2021-03-08 15:15:01.229,android.net.conn.INET_CONDITION_ACTION,NaN
2021-03-08 15:15:01.229,2021-03-08 15:15:01.229,android.net.nsd.STATE_CHANGED,NaN
2021-03-08 15:15:01.230,2021-03-08 15:15:01.230,android.net.conn.CONNECTIVITY_CHANGE,NaN


In [7]:
one_hot_maj_class = pd.get_dummies(df['major_class'], prefix='one_hot_c')
df = df.join(one_hot_maj_class)

one_hot_bond = pd.get_dummies(df['bond_state'], prefix='one_hot_b')
df = df.join(one_hot_bond)

one_hot_type = pd.get_dummies(df['type'], prefix='one_hot_t')
df = df.join(one_hot_type)

In [9]:
df = df.drop(['timestamp', 'ssid', 'class', 'major_class', 'bond_state', 'type'], axis = 1)

In [10]:
bssid_map = { bssid.replace(' ', ''): idx for bssid, idx in zip(df.bssid.unique(), range(len(df.bssid.unique()))) }

In [11]:
df.bssid = df.bssid.apply(lambda x: str(x).replace(' ', ''))

In [12]:
df['count'] = 1

In [13]:
def agg_string_join(col):
    col = col.apply(lambda x: str(x))
    return col.str.cat(sep = ',').replace(' ', '')

In [14]:
def agg_bssid_col(col):
    array_len = len(bssid_map)
    array = np.zeros(array_len, dtype = 'int8')
    def fill_array(bssid):
        array[bssid_map[bssid.replace(' ', '')]] = 1
        return
        
    col.apply(lambda x: fill_array(x))
    return np.array2string(array, separator = ',').replace(' ', '')[1:-1]

In [15]:
one_hot_columns_count = 0
for col in df.columns:
    if col.find('one_hot') != -1:
        one_hot_columns_count += 1

In [16]:
cat_columns = df.columns[1:1 + one_hot_columns_count]
cat_columns_map = { col: 'mean' for col in cat_columns }

all_func_dicts_quantum = { 'bssid' : agg_bssid_col, 'count' : 'sum' }
all_func_dicts_quantum.update(cat_columns_map)

In [17]:
df_quantum = df.groupby(pd.Grouper(freq = '5s'), as_index=True).agg(all_func_dicts_quantum)

In [18]:
df_quantum = df_quantum.reset_index()
df_quantum.index = pd.DatetimeIndex(df_quantum.timestamp)

In [19]:
df_quantum = df_quantum.dropna()

In [20]:
df_le = pd.read_csv(".\\_generated\\le_bt_3.data", index_col = False, header = None, low_memory = False, \
                 names = ['timestamp', 'stamp', '1', '2', 'level', '3', 'connectable', '8'])

df_le = df_le.drop(df_le.columns.difference(['connectable','timestamp', 'level']), axis = 1)
df_le['timestamp'] = df_le['timestamp'].apply(lambda x: dt.strptime(x, '%d.%m.%Y_%H:%M:%S.%f'))
df_le.index = pd.DatetimeIndex(df_le.timestamp)
df_le = df_le.sort_index()

In [21]:
df_le['connectable'] = df_le['connectable'].apply(lambda x: 1 if str(x).lower() == 'true' else 0)

In [22]:
df_le = df_le.groupby(pd.Grouper(freq = '5s'), as_index=True).agg({'level':'mean', 'connectable':'mean'})

In [23]:
df_le = df_le.dropna()

In [24]:
def get_le_conn_status_from_row(row):
    conn = df_le.iloc[df_le.index.get_loc(row.name, method = 'nearest')]['connectable']
    time = df_le.iloc[df_le.index.get_loc(row.name, method = 'nearest')].name
    return conn if abs((time - row.name).total_seconds()) < 10 else 0

def get_le_level_from_row(row):
    level = df_le.iloc[df_le.index.get_loc(row.name, method = 'nearest')]['level']
    time = df_le.iloc[df_le.index.get_loc(row.name, method = 'nearest')].name
    return level if abs((time - row.name).total_seconds()) < 10 else 0


df_quantum['le_connectable'] = df_quantum.apply(lambda row: get_le_conn_status_from_row(row), axis = 1)
df_quantum['le_level'] = df_quantum.apply(lambda row: get_le_level_from_row(row), axis = 1)

In [25]:
def string2array(string):
    try:
        array = np.fromstring(string, sep=',')
        return array
    except:
        return np.nan

def to_ones_array(array):
    try:
        array[array != 0] = 1
        return array
    except:
        return np.nan

def get_len(obj):
    try:
        length = len(obj)
        return length
    except:
        return np.nan

In [26]:
def get_occured_nets_count(row, prev_col, curr_col):
    prev = to_ones_array(string2array(row[prev_col]))
    curr = to_ones_array(string2array(row[curr_col]))
    intersection = np.logical_and(curr, prev)
    diff = np.logical_and(curr, np.logical_not(intersection))
    
    if (np.count_nonzero(np.logical_or(prev, curr)) == 0):
        return 0
    
    return np.count_nonzero(diff) / np.count_nonzero(np.logical_or(prev, curr))

def get_disappeared_nets_count(row, prev_col, curr_col):
    prev = to_ones_array(string2array(row[prev_col]))
    curr = to_ones_array(string2array(row[curr_col]))
    intersection = np.logical_and(curr, prev)
    diff = np.logical_and(prev, np.logical_not(intersection))
    
    if (np.count_nonzero(np.logical_or(prev, curr)) == 0):
        return 0
    
    return np.count_nonzero(diff) / np.count_nonzero(np.logical_or(prev, curr))

def get_jaccard_index(row, prev_col, curr_col):
    prev = to_ones_array(string2array(row[prev_col]))
    curr = to_ones_array(string2array(row[curr_col]))
    return distance.jaccard(prev, curr)

def get_occur_speed(row, prev_col, curr_col):
    prev = to_ones_array(string2array(row[prev_col]))
    curr = to_ones_array(string2array(row[curr_col]))
    return np.linalg.norm(prev - curr) / np.sqrt(get_len(prev))

def calc_single_cols_in_window(df, col, new_col, window, func):
    def func_wrapper(func, row, prev_col, curr_col):
        delta = row.timestamp - row.prev_timestamp
        if pd.isnull(delta):
            delta = 0
        else:
            delta = abs(delta.total_seconds())
        if delta > 10 * 60:
            return np.nan
        else:
            return func(row, prev_col_name, col)
        
    new_cols = []
        
    for i in range(window):
        prev_col_name = "_".join(['prev', col, str(i + 1)])
        new_col_name = "_".join([new_col, str(i + 1)])
        
        df.loc[:, 'prev_timestamp'] = df.timestamp.shift(i + 1)
        df.loc[:, prev_col_name] = df[col].shift(i + 1)
        df.loc[:, new_col_name] = df.apply(lambda row: func_wrapper(func, row, prev_col_name, col), axis = 1)
        df = df.drop(prev_col_name, axis = 1)
        df = df.drop('prev_timestamp', axis = 1)
        new_cols.append(new_col_name)
        
    df.loc[:, "_".join([new_col, 'mean'])] = df[new_cols].mean(axis = 1)
    df.loc[:, "_".join([new_col, 'median'])] = df[new_cols].median(axis = 1)
    df.loc[:, "_".join([new_col, 'var'])] = df[new_cols].var(axis = 1)
        
    return df

In [27]:
WINDOW_SIZE = 3

occur_and_level_columns_map = [
    ("bssid", "occured_devices_count", WINDOW_SIZE, get_occured_nets_count),
    ("bssid", "disappeared_devices_count", WINDOW_SIZE, get_disappeared_nets_count),
    ("bssid", "jaccard_index", WINDOW_SIZE, get_jaccard_index), 
    ("bssid", "occur_speed", WINDOW_SIZE, get_occur_speed)
]

for (col, new_col, window, func) in occur_and_level_columns_map:
    df_quantum = calc_single_cols_in_window(df_quantum, col, new_col, window, func)

In [28]:
def get_conn_level_speed(row, prev_col, curr_col):
    return row[curr_col] - row[prev_col]

In [29]:
WINDOW_SIZE = 3

single_columns_map = [
    ("count", "count_speed", WINDOW_SIZE, get_conn_level_speed)
]

for (col, new_col, window, func) in single_columns_map:
    df_quantum = calc_single_cols_in_window(df_quantum, col, new_col, window, func)

In [30]:
def get_acceleration(row, prev_col, curr_col):
    return abs(row[curr_col] - row[prev_col])

In [31]:
WINDOW_SIZE = 3

multi_speed_cols = ["occured_devices_count_mean", "jaccard_index_mean", "occur_speed_mean", "disappeared_devices_count_mean", 
                    "count_speed_mean"]

for col in multi_speed_cols:
    df_quantum = calc_single_cols_in_window(df_quantum, col, "_".join(["acceleration", col]), window, func)

In [32]:
def agg_str(col):
    all_freq = col.str.cat(sep=',')
    return string2array(all_freq)

def str_mean(col):
    return np.mean(agg_str(col))

def str_var(col):
    return np.var(agg_str(col))

def str_median(col):
    return np.median(agg_str(col))

def str_skew(col):
    return stats.skew(agg_str(col))

def str_kurt(col):
    return stats.kurtosis(agg_str(col))

def mean(col):
    return np.mean(col)

def var(col):
    return np.var(col)

def median(col):
    return np.median(col)

def skew(col):
    return stats.skew(col)

def kurt(col):
    return stats.kurtosis(col)

In [33]:
df_quantum = df_quantum.drop(['bssid', 'timestamp'], axis = 1)

In [34]:
common_cols = df_quantum.columns[:one_hot_columns_count + 3]
speed_acc_cols = df_quantum.columns[one_hot_columns_count + 3:]

common_funcs_list = [mean, var, median, skew, kurt]
special_funcs_list = [mean, pd.DataFrame.mad, skew]

common_cols_map = { col : common_funcs_list for col in common_cols }
speed_acc_cols_map = { col : special_funcs_list for col in speed_acc_cols }

agg_dict = common_cols_map
agg_dict.update(speed_acc_cols_map)

In [35]:
df_quantum[speed_acc_cols] = df_quantum[speed_acc_cols].apply(pd.to_numeric)

In [36]:
df_sampling = df_quantum.groupby(pd.Grouper(freq = TIME_SAMPLE_FREQ)).agg(agg_dict)

D:\Program Files\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\Program Files\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [37]:
df_rolling = df_quantum.rolling(TIME_SAMPLE_FREQ, min_periods = 1, center = False).agg(agg_dict)

In [38]:
df_sampling.columns = ["_".join([str(high_level_name), str(low_level_name)]) \
                       for (high_level_name, low_level_name) in df_sampling.columns.values]

df_rolling.columns = ["_".join([str(high_level_name), str(low_level_name)]) \
                      for (high_level_name, low_level_name) in df_rolling.columns.values]

In [39]:
df_sampling = df_sampling.dropna()
df_sampling = df_sampling.fillna(0)

df_rolling = df_rolling.dropna()
df_rolling = df_rolling.fillna(0)

In [40]:
# df_rolling['day_time'] = df_rolling.apply(lambda x: x.name.hour + x.name.minute / 60 , axis = 1)
# df_sampling['day_time'] = df_sampling.apply(lambda x: x.name.hour + x.name.minute / 60 , axis = 1)

In [41]:
df_sampling.to_csv(".\\_datasets\\5s\\bt_sampling_dataset_3.csv")
df_rolling.to_csv(".\\_datasets\\5s\\bt_rolling_dataset_3.csv")